In [1]:
!source ../../env/bin/activate

In [2]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [3]:
mission = "../../data/missions.csv"
mission_df = pd.read_csv(mission, index_col='ID').drop(columns=['Unnamed: 0'])

astronauts = "../../data/astronauts.csv"
astronauts_df = pd.read_csv(astronauts).drop(columns=['Unnamed: 0', 'id', 'original_name'])

In [4]:
astronauts_df['occupation'] = astronauts_df['occupation'].str.lower()
astronauts_df.loc[astronauts_df['occupation'] == 'space tourist', 'occupation'] = 'other (space tourist)'

In [5]:
missing_shuttles = astronauts_df.loc[astronauts_df['ascend_shuttle'].isna()].copy()
missing_shuttles.loc[:, ['ascend_shuttle', 'descend_shuttle']] = missing_shuttles['in_orbit'].item()
astronauts_df.loc[astronauts_df['ascend_shuttle'].isna()] = missing_shuttles

In [6]:
missing_mission_titles = astronauts_df.loc[astronauts_df['mission_title'].isna()].copy()
missing_mission_titles.loc[:, 'mission_title'] = 60
astronauts_df.loc[astronauts_df['mission_title'].isna()] = missing_mission_titles

In [7]:
missing_selection = astronauts_df.loc[astronauts_df['selection'].isna()].copy()
missing_selection.loc[:, 'selection'] = 'MirCorp'
astronauts_df.loc[astronauts_df['selection'].isna()] = missing_selection

In [8]:
mission_df['Cost'] = mission_df['Cost'].str.replace(',', '')
mission_df['Cost'] = pd.to_numeric(mission_df['Cost'])

In [9]:
mission_df.loc[mission_df['Company Name'] == "Arm??e de l'Air", ['Company Name']] = "Arme de l'Air"

In [10]:
mission_df['Country'] = mission_df['Location'].str.extract(r'^.*?([^\t,]*)$')
mission_df['Country'] = mission_df['Country'].str.strip()

In [11]:
mission_df[['Vehicle', 'Mission']] = mission_df['Detail'].str.split('|', expand=True)

In [12]:
mission_df['Status Rocket'] = mission_df['Status Rocket'].replace({'StatusActive':'Active', 'StatusRetired':'Retired'})

In [13]:
mission_df = mission_df.drop('Detail', axis=1)

In [14]:
with pd.ExcelWriter('../../data/cleaned_data.xlsx') as writer:  
    astronauts_df.to_excel(writer, sheet_name='astronauts', index=False)
    mission_df.to_excel(writer, sheet_name='missions', index=False)